In [19]:
%matplotlib inline
%load_ext tensorboard
#!pip install keras-tuner
import pandas as pd
import numpy as np
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.layers import Layer
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)
from matplotlib import pyplot as plt
import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.python.ops.numpy_ops import np_config

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [20]:
genotypeFile = 'genotype.csv'
genotype = pd.read_csv(genotypeFile, sep = '\t', index_col = 0)
print('genotypeFile shape:', genotype.shape )

phenotypeFile = 'phenotype.csv'
multi_pheno = pd.read_csv(phenotypeFile, sep = ',', index_col = 0)
print('Phenotype_Multi shape:', multi_pheno.shape )


# take a small part to test code
# genotype
X = genotype
#display(X.head())
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno

#Chage the index for each model here accoding to the correspinding model
Y = multi_pheno.iloc[:, 1]#index=1 --> QPPCNN_1_CopperSulfate_1
# Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones
# corrupted_X.head()

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
# corrupted_X_onehot.shape

# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

genotypeFile shape: (4390, 28220)
Phenotype_Multi shape: (4390, 20)


In [21]:
x_test.shape

(396, 28220, 3)

In [22]:
len(x_train[:100])

100

In [23]:
background_list = list([i for i in range(404)])[0:100]
if len(background_list)==0:
    background_list = x_train[np.random.choice(x_train.shape[0],100, replace=False)]
background = x_train[background_list]
background.shape

(100, 28220, 3)

In [24]:
import shap
from shap import initjs
initjs()
custom_objects = None
# Define custom objects if necessary
custom_objects = None

# Load the model with specified reduction strategy
model_mse = tf.keras.models.load_model('1_CopperSulfate_1.h5', custom_objects=custom_objects, compile=False)

The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
2024-07-02 09:11:49.005727: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_4_1/kernel/Assign' id:553 op device:{requested: '', assigned: ''} def:{{{node dense_4_1/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_4_1/kernel, dense_4_1/kernel/Initializer/stateless_truncated_normal)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [25]:
model_mse.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28220, 3)]        0         
                                                                 
 conv1d_2 (Conv1D)           (None, 9405, 8)           176       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 4702, 8)          0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 1566, 8)           456       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 783, 8)           0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 6264)              0   

In [26]:
explainer = shap.DeepExplainer(model_mse,x_test)
base_value = explainer.expected_value

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode. See PR #1483 for discussion.


In [27]:
base_values = base_value[0]

In [28]:
shap_values = explainer.shap_values(x_test,check_additivity=False)

In [29]:
mean_shap_values = np.mean(shap_values, axis=0)

In [30]:
abs_shap_values = np.abs(shap_values[0])
feature_importance = np.mean(abs_shap_values, axis=0)
len(feature_importance)

28220

In [31]:
feature_importance.shape

(28220, 3)

In [32]:
feature_impportance_sum = feature_importance.sum(axis=1)
feature_impportance_sum

array([0.00044224, 0.00025309, 0.00056062, ..., 0.        , 0.        ,
       0.        ])

In [33]:
feature_importance_df = pd.DataFrame({
    'Feature': X.columns.tolist(),  # Convert X.columns to a list
    'Importance': feature_impportance_sum.tolist()
})

#feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
feature_importance_df

,Feature,Importance
0,33070_chrI_33070_A_T,0.000442
1,33147_chrI_33147_G_T,0.000253
2,33152_chrI_33152_T_C,0.000561
3,33200_chrI_33200_C_T,0.000237
4,33293_chrI_33293_A_T,0.000178
...,...,...
28215,12051240_chrXVI_927980_A_G,0.000001
28216,12051367_chrXVI_928107_C_T,0.000000
28217,12052782_chrXVI_929522_C_T,0.000000
28218,12052988_chrXVI_929728_A_G,0.000000


In [34]:
num = feature_importance_df[feature_importance_df['Importance']  <= 1e-4]
less_important_features = num.Feature.tolist()
len(less_important_features)

1373

In [35]:
num = feature_importance_df[feature_importance_df['Importance']  >= 1e-4]
important_features = num.Feature.tolist()
important_features

['33070_chrI_33070_A_T',
 '33147_chrI_33147_G_T',
 '33152_chrI_33152_T_C',
 '33200_chrI_33200_C_T',
 '33293_chrI_33293_A_T',
 '33328_chrI_33328_C_A',
 '33348_chrI_33348_G_C',
 '33403_chrI_33403_C_T',
 '33502_chrI_33502_A_G',
 '33548_chrI_33548_A_C',
 '33892_chrI_33892_C_G',
 '33996_chrI_33996_A_G',
 '34170_chrI_34170_T_A',
 '34515_chrI_34515_T_C',
 '34554_chrI_34554_A_G',
 '34943_chrI_34943_C_G',
 '34991_chrI_34991_A_G',
 '35059_chrI_35059_G_A',
 '35070_chrI_35070_C_T',
 '35129_chrI_35129_G_A',
 '35466_chrI_35466_G_A',
 '35508_chrI_35508_A_G',
 '35565_chrI_35565_A_G',
 '35568_chrI_35568_A_G',
 '35631_chrI_35631_G_C',
 '35668_chrI_35668_G_A',
 '35736_chrI_35736_G_A',
 '35751_chrI_35751_T_C',
 '35818_chrI_35818_A_G',
 '35970_chrI_35970_T_C',
 '36005_chrI_36005_A_G',
 '36048_chrI_36048_C_G',
 '36123_chrI_36123_C_T',
 '36160_chrI_36160_G_A',
 '36313_chrI_36313_A_G',
 '36524_chrI_36524_C_T',
 '36574_chrI_36574_A_G',
 '36637_chrI_36637_A_G',
 '36898_chrI_36898_C_T',
 '37008_chrI_37008_C_A',


In [36]:
def find_indexes(main_list, search_list):
    return [i for i, x in enumerate(main_list) if x in search_list]

indexes_shap = find_indexes(feature_importance_df['Feature'].tolist(), important_features)
print(len(indexes_shap))

26847


In [ ]:
feature_importance_dict = dict(zip(feature_importance_df['Feature'], feature_importance_df['Importance']))
feature_importance_dict

In [ ]:
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

#### First we want to remove all values that are less than and equal to zero and save them in a list

removed_pairs_features = []

filtered_pairs_features = [(key, value) for key, value in sorted_features if value > 0]
removed_pairs_features = [(key, value) for key, value in sorted_features if value <= 0]
filtered_pairs_features_numpy = np.array(filtered_pairs_features)
# print("Filtered key-value pairs:", filtered_pairs_features)
# print("Removed key-value pairs:", removed_pairs_features)

In [ ]:
import matplotlib.pyplot as plt

# Extract feature names and importance values for visualization
sorted_feature_names = [x[0] for x in filtered_pairs_features[:20]]
sorted_feature_importance = [x[1] for x in filtered_pairs_features[:20]]

# Plot the ranked features
plt.figure(figsize=(10, 6))
plt.barh(sorted_feature_names, sorted_feature_importance)
plt.xlabel('SHAP Feature Importance')
plt.ylabel('Feature')
plt.title('Ranked Features based on SHAP Values')
plt.show()

In [ ]:
feature_shap_values = [pair[1] for pair in filtered_pairs_features[:30]]
feature_shap_values_names = [pair[0] for pair in filtered_pairs_features[:30]]
feature_names = np.array(feature_shap_values_names)
type(feature_shap_values)

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a list of features and their corresponding SHAP values
features = feature_importance_df['Feature'][:20]
shap_values = feature_importance_df['Importance'][:20]

# Plotting
plt.figure(figsize=(8, 6))
plt.scatter(shap_values, features, color='green', s=100, alpha=0.6)
plt.xlabel('SHAP Value')
plt.ylabel('Feature')
plt.title('Feature with their corresponding SHAP value')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
### Generating the FORCE Plot
shap_values = np.array(feature_shap_values)  # Assuming 100 samples and 5 features
shap_values_ = np.reshape(shap_values, (-1,2))
shap.force_plot(base_values, shap_values, feature_names)

In [ ]:
### Generating the Waterfall Plot

explanation = shap.Explanation(values=shap_values , base_values=base_values, feature_names = feature_names)

shap.waterfall_plot(explanation)

In [ ]:
genotype

In [ ]:
# Sample dataset with a categorical column
data = genotype
df_data = pd.DataFrame(data)
# Perform one-hot encoding on a specific categorical column
encoded_df = pd.get_dummies(df_data)

# Display the dataset after one-hot encoding
print("\nDataset after One-Hot Encoding:")
print(encoded_df)
encoded_df.shape

In [ ]:
import pandas as pd

# Sample one-hot encoded dataset
data = encoded_df
df_data = pd.DataFrame(data)

# Select features to set one-hot encoding to zero
selected_features = less_important_features

# Set one-hot encoding of selected features to zero
for feature in selected_features:
    for col in df_data.columns:
        if feature in col:
            df_data[col] = 0

# Check the modified dataset
print(df_data)

In [ ]:
print(df_data['10178337_chrXV_146368_C_T'])

In [ ]:
X = df_data
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno
Y = multi_pheno.iloc[:, 1]#index=1 --> 1_CopperSulfate_1
# Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones
print(corrupted_X['10178337_chrXV_146368_C_T'])

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
print(type(corrupted_X_onehot))
#print(corrupted_X['6810072_chrXI_231860_A_G'])
# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

In [ ]:
### **Retrain the model on the new dataset**

def model_build(pheno_name,num_hl,hl_list,hl_activation, out_activation, dropout_val,filters_, kernel_size_,stride_poolSize,Conv_Layers_Stride_Size):
    

    '''
    https://towardsdatascience.com/multi-output-model-with-tensorflow-keras-functional-api-875dd89aa7c6

    def DNN_build(num_hl,hl_list,hl_activation, out_activation, dropout_val):
    pheno_name = name of phenotype of interest
    num_hl = number of hidden layers
    hl_list = list of hidden layers
    hl_activation = hidden layer activation function
    out_activation = output layer activation function
    dropout_val = Dropout value
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    input_layer = tf.keras.Input(shape=(28220,3))#(3138, 28220, 3)
    
    shared_convL1 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation,input_shape = (28220,3))(input_layer)
    shared_convL1_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL1)
    
    shared_convL2 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation)(shared_convL1_max_pool)
    shared_convL2_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL2)
    
    shared_convLayer_Flatten = tf.keras.layers.Flatten()(shared_convL2_max_pool)
    initializer = tf.keras.initializers.HeNormal()
    kernel_regularizer_ = tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001)
    
    shared_hidden_layer1 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[0], activation  = hl_activation)(shared_convLayer_Flatten)
    shared_hidden_layer1_dp1 = tf.keras.layers.Dropout(dropout_val[0])(shared_hidden_layer1)

    #model 1
    model1_hidden_layer2 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[1], activation  = hl_activation)(shared_hidden_layer1_dp1)
    model1_hidden_layer2_dp2 = tf.keras.layers.Dropout(dropout_val[1])(model1_hidden_layer2)

    model1_hidden_layer3 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[2], activation  = hl_activation)(model1_hidden_layer2_dp2)
    model1_hidden_layer3_dp3 = tf.keras.layers.Dropout(dropout_val[2])(model1_hidden_layer3)

    model1_hidden_layer4 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[3], activation  = hl_activation)(model1_hidden_layer3_dp3)
    model1_hidden_layer4_dp4 = tf.keras.layers.Dropout(dropout_val[3])(model1_hidden_layer4)
    model1_hidden_layer4_dp4_fl = tf.keras.layers.Flatten()(model1_hidden_layer4_dp4)
    
    pheno_name_ = tf.keras.layers.Dense(units = 1, name = pheno_name, activation  = out_activation)(model1_hidden_layer4_dp4_fl)#1_CobaltChloride_1
    model = tf.keras.models.Model(inputs = input_layer, outputs = [pheno_name_])

    return model



In [ ]:
#ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy','mse'])#change metrics to MSE
def compile_model(pheno_name,model,loss_, learningRate, metrics_): #loss = 'sparse_categorical_crossentropy'
    '''
    def compile_model(DNN, loss_, learningRate, metrics_):
    DNN: the model
    loss_: the loss function
    learningRate: learning rate
    metrics_: metrics of interest ['accuracy', 'mse']
    '''

    # Specify the optimizer, and compile the model with loss functions for both outputs
    model.compile(
       #optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate),#0.00027705 
    optimizer =  tf.keras.optimizers.legacy.SGD(learning_rate=learningRate, momentum=0.01, nesterov=True, name="SGD"),
      loss = {
          pheno_name:loss_
      }, 
      metrics = [metrics_]#{'CobaltChloride_1':[metrics_],'CopperSulfate_1':[metrics_],'Diamide_1':[metrics_]}
      )#change metrics to MSE ##['accuracy','mse']
    return model

def runModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode,pheno_index):
    '''
    def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
    pheno_index: the index of the phenotype in y_train
    '''
    history = model.fit(
    x_train, 
    [y_train[:]],
    validation_data=(x_valid, [y_valid[:]]),
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )

    return history



In [ ]:
def runModel_all(pheno_name, pheno_index,hl_list, drop_list):
    model_mse = model_build(pheno_name,4,hl_list,'relu','linear',drop_list,8,7,2,3)#model initiallization
    model_mse = compile_model(pheno_name,model_mse,tf.keras.losses.MeanSquaredError(reduction="auto", name="mse"),
    0.00123695244,'mse')
    history_mse =  runModel(model_mse,0.30,32,400,15, "val_mse", "min",pheno_index)
    return history_mse,model_mse

In [ ]:
history_mse,model_mse = runModel_all('1_CopperSulfate_1',1,[1464,608,264,146],[0.00,0.00,0.00,0.20])

In [ ]:
model_mse.save('important features/saved Models/1_CopperSulfate_1.h5')

In [ ]:
from sklearn.metrics import mean_squared_error

# Make predictions on the validation set
y_pred = model_mse.predict(x_valid)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_valid, y_pred)
print("Mean Squared Error on the validation set:", mse)

# Make predictions on the tesing set
y_pred = model_mse.predict(x_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on the testing set:", mse)



In [37]:
#### This is for LIME
import lime
import lime.lime_tabular
model_mse= tf.keras.models.load_model('1_CopperSulfate_1.h5')
print(type(genotype))
genotype_data = genotype.to_numpy()
class_names = ['Positive','Negative']
my_features = genotype.columns.tolist()


explainer = lime.lime_tabular.RecurrentTabularExplainer(x_train, mode='regression', training_labels=None, feature_names=my_features, 
                        categorical_features=None, categorical_names=None, kernel_width=None, kernel=None, verbose=False, 
                        class_names=class_names, feature_selection='auto', discretize_continuous=True, discretizer='quartile', 
                        random_state=None)


The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
2024-07-02 09:30:28.338649: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_7_2/kernel/Assign' id:1096 op device:{requested: '', assigned: ''} def:{{{node dense_7_2/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_7_2/kernel, dense_7_2/kernel/Initializer/stateless_truncated_normal)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


<class 'pandas.core.frame.DataFrame'>


In [38]:
i = 0
instance = x_test[i]
explanation = explainer.explain_instance(instance, model_mse.predict, num_features=len(my_features))

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
2024-07-02 09:38:06.762391: W tensorflow/c/c_api.cc:300] Operation '{name:'1_CopperSulfate_1_2/BiasAdd' id:1141 op device:{requested: '', assigned: ''} def:{{{node 1_CopperSulfate_1_2/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](1_CopperSulfate_1_2/MatMul, 1_CopperSulfate_1_2/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-07-02 09:38:06.913669: W tensorflow/c/c_api.cc:300] Operation '{name:'total/Assign' id:1212 op device:{requested: '', assigned: ''} def:{{{node total/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](total, total/Initializer/zero

In [39]:
explanation.as_list()
lime_importance = dict(explanation.as_list())
lime_importance

{'0.00 < 33147_chrI_33147_G_T_t-13379 <= 1.00': 0.0010216382551241703,
 '0.00 < 33152_chrI_33152_T_C_t-26254 <= 1.00': 0.0009836326989487705,
 '0.00 < 33152_chrI_33152_T_C_t-23830 <= 1.00': -0.000968729801957779,
 '0.00 < 33147_chrI_33147_G_T_t-8551 <= 1.00': -0.0009630850481592414,
 '0.00 < 33147_chrI_33147_G_T_t-4811 <= 1.00': -0.0009607517546959002,
 '0.00 < 33147_chrI_33147_G_T_t-12364 <= 1.00': -0.000958895977577082,
 '0.00 < 33147_chrI_33147_G_T_t-25764 <= 1.00': 0.0009490279379400587,
 '33147_chrI_33147_G_T_t-11054 <= 0.00': -0.0009456360553019821,
 '33147_chrI_33147_G_T_t-21951 <= 0.00': -0.0009361635664304362,
 '0.00 < 33147_chrI_33147_G_T_t-15157 <= 1.00': 0.0009361062572191358,
 '0.00 < 33147_chrI_33147_G_T_t-7179 <= 1.00': -0.0009260894518630743,
 '33147_chrI_33147_G_T_t-19628 <= 0.00': -0.0009225019627568612,
 '33152_chrI_33152_T_C_t-4421 <= 0.00': -0.0009220258267675878,
 '0.00 < 33152_chrI_33152_T_C_t-24874 <= 1.00': 0.000914918427139918,
 '33152_chrI_33152_T_C_t-8948 <=

In [ ]:
sorted_lime_importance = dict(sorted(lime_importance.items(), key=lambda item: item[1], reverse=True))
sorted_lime_importance

In [40]:
less_important_lime = [k for k, v in lime_importance.items() if v <= 1e-5]
len(less_important_lime)

15602

In [41]:
important_lime = [k for k, v in lime_importance.items() if v >= 1e-5]
len(important_lime)

12618

In [42]:
def find_indexes(main_list, search_list):
    return [i for i, x in enumerate(main_list) if x in search_list]

indexes_lime = find_indexes(lime_importance, less_important_lime)
print(indexes_lime) 

[2, 3, 4, 5, 7, 8, 10, 11, 12, 15, 18, 19, 21, 22, 23, 26, 27, 28, 31, 35, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 52, 53, 55, 58, 61, 63, 64, 66, 67, 69, 70, 72, 74, 76, 77, 78, 79, 81, 82, 85, 86, 88, 89, 90, 91, 92, 93, 94, 96, 98, 99, 103, 104, 105, 106, 107, 108, 109, 111, 112, 114, 116, 118, 119, 121, 122, 125, 126, 127, 128, 129, 130, 131, 133, 135, 137, 138, 139, 140, 142, 144, 145, 147, 148, 149, 150, 152, 153, 154, 155, 156, 159, 160, 161, 164, 165, 166, 168, 169, 170, 172, 174, 176, 177, 178, 179, 180, 181, 182, 186, 187, 188, 191, 192, 196, 198, 199, 200, 201, 203, 205, 206, 207, 208, 209, 211, 213, 214, 216, 218, 219, 222, 224, 225, 226, 229, 231, 232, 234, 235, 236, 237, 238, 243, 246, 247, 248, 249, 257, 258, 259, 260, 261, 263, 264, 266, 267, 268, 269, 270, 273, 274, 276, 278, 279, 282, 285, 287, 288, 289, 290, 293, 295, 296, 297, 301, 303, 304, 305, 306, 309, 311, 312, 314, 315, 316, 317, 319, 320, 321, 322, 324, 325, 326, 327, 328, 329, 330, 334, 335, 337, 339, 34

In [ ]:
import matplotlib.pyplot as plt

sorted_lime_importance_list = sorted(sorted_lime_importance.items(), key=lambda item: item[1], reverse=True)

num_features_to_plot = min(20, len(sorted_lime_importance_list))

# Extract feature names and importance values for visualization
sorted_lime_names = [x[0] for x in sorted_lime_importance_list[:num_features_to_plot]]
sorted_lime_importance_ = [x[1] for x in sorted_lime_importance_list[:num_features_to_plot]]

# Plot the ranked features
plt.figure(figsize=(10, 6))
plt.barh(sorted_lime_names, sorted_lime_importance_)
plt.xlabel('LIME Feature Importance')
plt.ylabel('Feature')
plt.title('Ranked Features based on LIME Values')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

sorted_lime_importance_list = sorted(sorted_lime_importance.items(), key=lambda item: item[1], reverse=True)

num_features_to_plot = min(20, len(sorted_lime_importance_list))

# Extract feature names and importance values for visualization
sorted_lime_names = [x[0] for x in sorted_lime_importance_list[:num_features_to_plot]]
sorted_lime_importance_ = [x[1] for x in sorted_lime_importance_list[:num_features_to_plot]]
# Plotting
plt.figure(figsize=(8, 6))
plt.scatter(sorted_lime_importance_, sorted_lime_names, color='green', s=100, alpha=0.6)
plt.xlabel('LIME Value')
plt.ylabel('Feature')
plt.title('Feature with their corresponding SHAP value')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
# Sample dataset with a categorical column
data_lime = genotype
df_data = pd.DataFrame(data_lime)
# Perform one-hot encoding on a specific categorical column
encoded_df = pd.get_dummies(df_data)

# Display the dataset after one-hot encoding
print("\nDataset after One-Hot Encoding:")
print(encoded_df)
encoded_df.shape

In [ ]:
import pandas as pd

# Sample one-hot encoded dataset
data = encoded_df
df_data_ = pd.DataFrame(data)
type(df_data.columns)
# Select features to set one-hot encoding to zero
selected_features = indexes

# Set one-hot encoding of selected features to zero
for index in selected_features:
    for col in df_data_.columns:
        if index in df_data_.columns:
            df_data_[col] = 0

# Check the modified dataset
print(df_data_)

In [ ]:
X = df_data_
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno
Y = multi_pheno.iloc[:, 1]#index=1 --> 1_CopperSulfate_1
# Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
print(type(corrupted_X_onehot))
#print(corrupted_X['6810072_chrXI_231860_A_G'])
# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

In [ ]:
### **Retrain the model on the new dataset**

def model_build(pheno_name,num_hl,hl_list,hl_activation, out_activation, dropout_val,filters_, kernel_size_,stride_poolSize,Conv_Layers_Stride_Size):
    

    '''
    https://towardsdatascience.com/multi-output-model-with-tensorflow-keras-functional-api-875dd89aa7c6

    def DNN_build(num_hl,hl_list,hl_activation, out_activation, dropout_val):
    pheno_name = name of phenotype of interest
    num_hl = number of hidden layers
    hl_list = list of hidden layers
    hl_activation = hidden layer activation function
    out_activation = output layer activation function
    dropout_val = Dropout value
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    input_layer = tf.keras.Input(shape=(28220,3))#(3138, 28220, 3)
    
    shared_convL1 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation,input_shape = (28220,3))(input_layer)
    shared_convL1_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL1)
    
    shared_convL2 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation)(shared_convL1_max_pool)
    shared_convL2_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL2)
    
    shared_convLayer_Flatten = tf.keras.layers.Flatten()(shared_convL2_max_pool)
    initializer = tf.keras.initializers.HeNormal()
    kernel_regularizer_ = tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001)
    
    shared_hidden_layer1 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[0], activation  = hl_activation)(shared_convLayer_Flatten)
    shared_hidden_layer1_dp1 = tf.keras.layers.Dropout(dropout_val[0])(shared_hidden_layer1)

    #model 1
    model1_hidden_layer2 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[1], activation  = hl_activation)(shared_hidden_layer1_dp1)
    model1_hidden_layer2_dp2 = tf.keras.layers.Dropout(dropout_val[1])(model1_hidden_layer2)

    model1_hidden_layer3 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[2], activation  = hl_activation)(model1_hidden_layer2_dp2)
    model1_hidden_layer3_dp3 = tf.keras.layers.Dropout(dropout_val[2])(model1_hidden_layer3)

    model1_hidden_layer4 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[3], activation  = hl_activation)(model1_hidden_layer3_dp3)
    model1_hidden_layer4_dp4 = tf.keras.layers.Dropout(dropout_val[3])(model1_hidden_layer4)
    model1_hidden_layer4_dp4_fl = tf.keras.layers.Flatten()(model1_hidden_layer4_dp4)
    
    pheno_name_ = tf.keras.layers.Dense(units = 1, name = pheno_name, activation  = out_activation)(model1_hidden_layer4_dp4_fl)#1_CobaltChloride_1
    model = tf.keras.models.Model(inputs = input_layer, outputs = [pheno_name_])

    return model



In [ ]:
#ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy','mse'])#change metrics to MSE
def compile_model(pheno_name,model,loss_, learningRate, metrics_): #loss = 'sparse_categorical_crossentropy'
    '''
    def compile_model(DNN, loss_, learningRate, metrics_):
    DNN: the model
    loss_: the loss function
    learningRate: learning rate
    metrics_: metrics of interest ['accuracy', 'mse']
    '''

    # Specify the optimizer, and compile the model with loss functions for both outputs
    model.compile(
       #optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate),#0.00027705 
    optimizer =  tf.keras.optimizers.legacy.SGD(learning_rate=learningRate, momentum=0.01, nesterov=True, name="SGD"),
      loss = {
          pheno_name:loss_
      }, 
      metrics = [metrics_]#{'CobaltChloride_1':[metrics_],'CopperSulfate_1':[metrics_],'Diamide_1':[metrics_]}
      )#change metrics to MSE ##['accuracy','mse']
    return model

def runModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode,pheno_index):
    '''
    def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
    pheno_index: the index of the phenotype in y_train
    '''
    history = model.fit(
    x_train, 
    [y_train[:]],
    validation_data=(x_valid, [y_valid[:]]),
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )

    return history



In [ ]:
def runModel_all(pheno_name, pheno_index,hl_list, drop_list):
    model_mse = model_build(pheno_name,4,hl_list,'relu','linear',drop_list,8,7,2,3)#model initiallization
    model_mse = compile_model(pheno_name,model_mse,tf.keras.losses.MeanSquaredError(reduction="auto", name="mse"),
    0.00123695244,'mse')
    history_mse =  runModel(model_mse,0.30,32,400,15, "val_mse", "min",pheno_index)
    return history_mse,model_mse

In [ ]:
history_mse,model_mse = runModel_all('1_CopperSulfate_1',1,[1464,608,264,146],[0.00,0.00,0.00,0.20])

In [ ]:
from sklearn.metrics import mean_squared_error

# Make predictions on the validation set
y_pred = model_mse.predict(x_valid)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_valid, y_pred)
print("Mean Squared Error on the validation set:", mse)

# Make predictions on the tesing set
y_pred = model_mse.predict(x_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on the testing set:", mse)


In [ ]:
explanation.show_in_notebook(show_table=True)

In [ ]:
###### Intersection of LIME and SHAP 

In [43]:
def find_common_indexes(lime_indexes, shap_indexes):
    # Convert lists to sets
    lime_set = set(lime_indexes)
    shap_set = set(shap_indexes)
    
    # Find the intersection
    common_indexes = lime_set.intersection(shap_set)
    
    return common_indexes
    

common_indexes = find_common_indexes(indexes_lime, indexes_shap)
print(type(common_indexes))



<class 'set'>


In [44]:
common_indexes = np.array(list(common_indexes))
(common_indexes)

array([    2,     3,     4, ..., 28191, 28192, 28193])

In [45]:
common_indexes_ = common_indexes.tolist()
len(common_indexes_)

14852

In [46]:
def get_strings_at_indices(strings, indices):

    valid_indices = [index for index in indices if 0 <= index < len(strings)]
    
    # Get strings at valid indices
    result = [strings[index] for index in valid_indices]
    
    return result

matching_features = get_strings_at_indices(feature_importance_df['Feature'].tolist(), common_indexes_)
len(matching_features)

14852

In [47]:
genotypeFile = 'genotype.csv'
genotype = pd.read_csv(genotypeFile, sep = '\t', index_col = 0)
print('genotypeFile shape:', genotype.shape )

phenotypeFile = 'phenotype.csv'
multi_pheno = pd.read_csv(phenotypeFile, sep = ',', index_col = 0)
print('Phenotype_Multi shape:', multi_pheno.shape )


# take a small part to test code
# genotype
X = genotype

mask = ~X.columns.str.contains('|'.join(matching_features))

# Set values to zero for non-matching columns
X.loc[:, mask] = 0

#display(X.head())
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno

#Chage the index for each model here accoding to the correspinding model
Y = multi_pheno.iloc[:, 1]#index=0 --> 1_CopperSulfate_1
# # Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones
# corrupted_X.head()

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
# corrupted_X_onehot.shape

# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

genotypeFile shape: (4390, 28220)
Phenotype_Multi shape: (4390, 20)


In [48]:
### **Retrain the model on the new dataset**

def model_build(pheno_name,num_hl,hl_list,hl_activation, out_activation, dropout_val,filters_, kernel_size_,stride_poolSize,Conv_Layers_Stride_Size):
    

    '''
    https://towardsdatascience.com/multi-output-model-with-tensorflow-keras-functional-api-875dd89aa7c6

    def DNN_build(num_hl,hl_list,hl_activation, out_activation, dropout_val):
    pheno_name = name of phenotype of interest
    num_hl = number of hidden layers
    hl_list = list of hidden layers
    hl_activation = hidden layer activation function
    out_activation = output layer activation function
    dropout_val = Dropout value
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    input_layer = tf.keras.Input(shape=(28220,3))#(3138, 28220, 3)
    
    shared_convL1 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation,input_shape = (28220,3))(input_layer)
    shared_convL1_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL1)
    
    shared_convL2 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation)(shared_convL1_max_pool)
    shared_convL2_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL2)
    
    shared_convLayer_Flatten = tf.keras.layers.Flatten()(shared_convL2_max_pool)
    initializer = tf.keras.initializers.HeNormal()
    kernel_regularizer_ = tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001)
    
    shared_hidden_layer1 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[0], activation  = hl_activation)(shared_convLayer_Flatten)
    shared_hidden_layer1_dp1 = tf.keras.layers.Dropout(dropout_val[0])(shared_hidden_layer1)

    #model 1
    model1_hidden_layer2 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[1], activation  = hl_activation)(shared_hidden_layer1_dp1)
    model1_hidden_layer2_dp2 = tf.keras.layers.Dropout(dropout_val[1])(model1_hidden_layer2)

    model1_hidden_layer3 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[2], activation  = hl_activation)(model1_hidden_layer2_dp2)
    model1_hidden_layer3_dp3 = tf.keras.layers.Dropout(dropout_val[2])(model1_hidden_layer3)

    model1_hidden_layer4 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[3], activation  = hl_activation)(model1_hidden_layer3_dp3)
    model1_hidden_layer4_dp4 = tf.keras.layers.Dropout(dropout_val[3])(model1_hidden_layer4)
    model1_hidden_layer4_dp4_fl = tf.keras.layers.Flatten()(model1_hidden_layer4_dp4)
    
    pheno_name_ = tf.keras.layers.Dense(units = 1, name = pheno_name, activation  = out_activation)(model1_hidden_layer4_dp4_fl)#1_CobaltChloride_1
    model = tf.keras.models.Model(inputs = input_layer, outputs = [pheno_name_])

    return model



In [49]:
#ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy','mse'])#change metrics to MSE
def compile_model(pheno_name,model,loss_, learningRate, metrics_): #loss = 'sparse_categorical_crossentropy'
    '''
    def compile_model(DNN, loss_, learningRate, metrics_):
    DNN: the model
    loss_: the loss function
    learningRate: learning rate
    metrics_: metrics of interest ['accuracy', 'mse']
    '''

    # Specify the optimizer, and compile the model with loss functions for both outputs
    model.compile(
       #optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate),#0.00027705 
    optimizer =  tf.keras.optimizers.legacy.SGD(learning_rate=learningRate, momentum=0.01, nesterov=True, name="SGD"),
      loss = {
          pheno_name:loss_
      }, 
      metrics = [metrics_]#{'CobaltChloride_1':[metrics_],'CopperSulfate_1':[metrics_],'Diamide_1':[metrics_]}
      )#change metrics to MSE ##['accuracy','mse']
    return model

def runModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode,pheno_index):
    '''
    def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
    pheno_index: the index of the phenotype in y_train
    '''
    history = model.fit(
    x_train, 
    [y_train[:]],
    validation_data=(x_valid, [y_valid[:]]),
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )

    return history



In [50]:
def runModel_all(pheno_name, pheno_index,hl_list, drop_list):
    model_mse = model_build(pheno_name,4,hl_list,'relu','linear',drop_list,8,7,2,3)#model initiallization
    model_mse = compile_model(pheno_name,model_mse,tf.keras.losses.MeanSquaredError(reduction="auto", name="mse"),
    0.00123695244,'mse')
    history_mse =  runModel(model_mse,0.30,32,400,15, "val_mse", "min",pheno_index)
    return history_mse,model_mse

In [53]:
history_mse,model_mse = runModel_all('1_CopperSulfate_1',1,[1464,608,264,146],[0.00,0.00,0.00,0.20])

The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.


Train on 3249 samples, validate on 369 samples


2024-07-02 10:14:44.486016: W tensorflow/c/c_api.cc:300] Operation '{name:'training_2/SGD/momentum/Assign' id:2745 op device:{requested: '', assigned: ''} def:{{{node training_2/SGD/momentum/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training_2/SGD/momentum, training_2/SGD/momentum/Initializer/initial_value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Epoch 1/400
3232/3249 [============================>.] - ETA: 0s - loss: 17.7376 - mse: 0.0486

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
2024-07-02 10:14:46.551060: W tensorflow/c/c_api.cc:300] Operation '{name:'loss_2/AddN_1' id:2545 op device:{requested: '', assigned: ''} def:{{{node loss_2/AddN_1}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss_2/mul, loss_2/AddN)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


3249/3249 [==============================] - 2s 542us/sample - loss: 17.7375 - mse: 0.0485 - val_loss: 17.6943 - val_mse: 0.0124
Epoch 2/400
3249/3249 [==============================] - 1s 428us/sample - loss: 17.6972 - mse: 0.0222 - val_loss: 17.6781 - val_mse: 0.0103
Epoch 3/400
3249/3249 [==============================] - 1s 426us/sample - loss: 17.6789 - mse: 0.0180 - val_loss: 17.6634 - val_mse: 0.0095
Epoch 4/400
3249/3249 [==============================] - 1s 428us/sample - loss: 17.6626 - mse: 0.0156 - val_loss: 17.6486 - val_mse: 0.0086
Epoch 5/400
3249/3249 [==============================] - 1s 425us/sample - loss: 17.6466 - mse: 0.0136 - val_loss: 17.6341 - val_mse: 0.0082
Epoch 6/400
3249/3249 [==============================] - 1s 422us/sample - loss: 17.6317 - mse: 0.0126 - val_loss: 17.6196 - val_mse: 0.0075
Epoch 7/400
3249/3249 [==============================] - 1s 422us/sample - loss: 17.6171 - mse: 0.0119 - val_loss: 17.6054 - val_mse: 0.0073
Epoch 8/400
3249/3249 [==

In [54]:
from sklearn.metrics import mean_squared_error

# Make predictions on the validation set
y_pred = model_mse.predict(x_valid)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_valid, y_pred)
print("Mean Squared Error on the validation set:", mse)

# Make predictions on the tesing set
y_pred = model_mse.predict(x_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on the testing set:", mse)


`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
2024-07-02 10:25:11.826483: W tensorflow/c/c_api.cc:300] Operation '{name:'1_CopperSulfate_1_3/BiasAdd' id:2396 op device:{requested: '', assigned: ''} def:{{{node 1_CopperSulfate_1_3/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](1_CopperSulfate_1_3/MatMul, 1_CopperSulfate_1_3/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Mean Squared Error on the validation set: 0.004289747965659345
Mean Squared Error on the testing set: 0.003954683763843939
